# Setting

In [36]:
# home path
import sys, os
home_path = os.path.expanduser("~") + '/Desktop/Molecular_kaggle'
sys.path.append(home_path + '/py/')

In [45]:
# module
import numpy as np
import pandas as pd
import pandas_profiling as pdp
import scipy
import lightgbm, xgboost
from tabulate import tabulate
from ggplot import *
import warnings
import collections
from scipy.optimize import minimize
from sympy import *
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() #seabornライブラリを読み込み、スタイルをセットする
from numpy import linalg as la
from sklearn.model_selection import StratifiedKFold, KFold

# my module 
import importlib
import boosting as gb
import base
importlib.reload(gb)
importlib.reload(base)

# for instance
Process = base.Process()

## Read csv

In [46]:
train, test, features = Process.read_data()

4658147 observations and 21 features in train set.
2505542 observations and 20 features in test set.
15 observations and 1 features in features set.


# boosting

In [47]:
# validation method
Sfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=831)
Kfolds = KFold(n_splits=5, shuffle=True, random_state=831)
# parameters
param_lgb1 = Process.open_parameter(file_name = 'kernel_param_lgb')
param_lgb1

{'num_leaves': 31, 'min_data_in_leaf': 30, 'objective': 'regression', 'max_depth': -1, 'learning_rate': 0.01, 'min_child_samples': 20, 'boosting': 'gbdt', 'feature_fraction': 0.9, 'bagging_freq': 1, 'bagging_fraction': 0.9, 'bagging_seed': 11, 'metric': 'rmse', 'lambda_l1': 0.1, 'verbosity': -1, 'nthread': 4, 'random_state': 831}


In [49]:
# model
with base.timer("Light GBM model"):
    lgb_val_pred, lgb_test_pred, lgb_importance = \
    gb.GradientBoosting(algorithm = gb.Lightgbm,
                        param_set = param_lgb1,
                        train = train, 
                        test = test,
                        features = features,
                        target = train["scalar_coupling_constant"],
                        folds = Sfolds)

fold n°1
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 13.4858	valid_1's rmse: 13.5077
[200]	training's rmse: 6.36877	valid_1's rmse: 6.37889
[300]	training's rmse: 4.58745	valid_1's rmse: 4.58818
[400]	training's rmse: 4.26322	valid_1's rmse: 4.25978
[500]	training's rmse: 4.20062	valid_1's rmse: 4.19639
[600]	training's rmse: 4.18084	valid_1's rmse: 4.17693
[700]	training's rmse: 4.16875	valid_1's rmse: 4.1652
[800]	training's rmse: 4.15953	valid_1's rmse: 4.15655
[900]	training's rmse: 4.15174	valid_1's rmse: 4.1491
[1000]	training's rmse: 4.14577	valid_1's rmse: 4.14369
[1100]	training's rmse: 4.14084	valid_1's rmse: 4.1393
[1200]	training's rmse: 4.13664	valid_1's rmse: 4.13558
[1300]	training's rmse: 4.1327	valid_1's rmse: 4.13224
[1400]	training's rmse: 4.12937	valid_1's rmse: 4.12934
[1500]	training's rmse: 4.12648	valid_1's rmse: 4.12696
[1600]	training's rmse: 4.12343	valid_1's rmse: 4.12434
[1700]	training's rmse: 4.12056	valid_1's rms

KeyboardInterrupt: 

In [ ]:
lgb_val_pred, lgb_test_pred, lgb_importance = val_pred, test_pred, importance

In [ ]:
# result
print("Lightgbm normal validation RMSE: %.4f" % np.sqrt(mean_squared_error(train["scalar_coupling_constant"].values, val_pred)))
# submit
Process.submit(lgb_test_pred,tech = "lightgbm_sample_output")

In [ ]:
# importance 
Process.display_importances(lgb_importance,title="LightGBM normal importance",file_name = None)